# ONTOP-Dremio Playground


**NOTE**: this notebook is written in JavaScript, so you need Jupyter + a JavaScript interpreter to run it. You may use the Deno interpreter, which is powerful and comes with a prebuild support for notebooks
```bash
curl -fsSL https://deno.land/install.sh | sh  # Install Deno
"$HOME/.deno/bin/deno" jupyter --install      # Add Deno to Jupyter
```
now just restart VSCode/Jupyter and you should see Deno amongst the kernels

In [1]:
const hello = "hello world!"
console.log(hello)

hello world!


In [5]:
// start docker compose if not already
const cmd = new Deno.Command("docker", { args: ["compose", "up", "-d"], stdout: "piped", stderr: "piped",}); 
const { success, stdout, stderr } = await cmd.output();
if (success) {
  console.log("Docker Compose started successfully!");
  console.log(new TextDecoder().decode(stdout));
} else {
  console.error(new TextDecoder().decode(stderr));
}

Docker Compose started successfully!



## Setup

In [7]:
import * as u from '../utils/index.ts'

// Connect to postgres
const pg1 = await u.pgCreateClient(5531)
const pg2 = await u.pgCreateClient(5532)
const pg3 = await u.pgCreateClient(5533)

// Connect to dremio and add postgres sources if not already there
// IMPORTANT: if it's the first time running, it may fail. Just run it again
const drm = await u.drmCreateClient()
try {
  await Promise.all([
    u.drmSetupPostgres(drm,"postgres1", "postgres1"),
    u.drmSetupPostgres(drm, "postgres2", "postgres2"),
    u.drmSetupPostgres(drm, "postgres3", "postgres3"),
  ]);
}catch(e){
  if(e.httpStatus === 409) console.log("Postgres sources already there - status 409")
  else throw e
}


// Check if everything is up
console.log('')
await u.pgLogTables(pg1)
console.log(await u.drmApi(drm, 'GET', '/catalog'))

User alredy there, no need to create a new one
Auth token: nlbjpqi7ovn15f86gtgmkafq1n - expires at: 17:46:38 (30hours)
Postgres sources already there - status 409

Tables: []
{
  data: [
    {
      id: "b7880582-c5c3-4be1-8fa5-0a275d83eb34",
      path: [ "@dremioUser" ],
      tag: "HE9xZ0EkRJc=",
      type: "CONTAINER",
      containerType: "HOME",
      createdAt: "2024-09-13T09:46:26.944Z"
    },
    {
      id: "0db052db-7a21-4e3e-ab9f-656abeae49fe",
      path: [ "postgres1" ],
      tag: "ttEIekt3CZI=",
      type: "CONTAINER",
      containerType: "SOURCE",
      createdAt: "2024-09-13T09:46:27.311Z"
    }
  ]
}


In [8]:
u.settings.debug = true

await u.pgClear(pg1, pg2, pg3)
await u.pgLoadSQLData(pg1, './data/postgres/hospital.sql')
await u.pgLoadSQLData(pg2, './data/postgres/patients.sql')
await u.pgLogTables(pg1)
await u.pgLogTables(pg2)


console.log(await u.ontopRunQuery({}, `
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX : <http://example.org/> 

    SELECT * WHERE {
      ?sub a :Patient.
    } 
  
    `))
    

Tables: [ { table_name: "patients" }, { table_name: "blood_tests" } ]
Tables: [ { table_name: "patients" } ]
09:48:27.264 |-WARN  in it.unibz.inf.ontop.cli.utils.EnvCli - Ignoring environment variable ONTOP_MAPPING_FILE=/opt/ontop/input/1/mapping.ttl for explicitly specified option -m
09:48:28.547 |-INFO  in com.dremio.jdbc.Driver - Enabling Netty native memory API for Java9+
09:48:28.790 |-INFO  in c.c.dremio.common.config.SabotConfig - Configuration and plugin file(s) identified in 75ms.
Base Configuration:
	- jar:file:/opt/ontop/jdbc/driver-gitexclude.jar!/sabot-default.conf

Intermediate Configuration and Plugin files, in order of precedence:
	- jar:file:/opt/ontop/jdbc/driver-gitexclude.jar!/sabot-module.conf


09:48:28.900 |-INFO  in c.o.a.arrow.memory.BaseAllocator - Debug mode disabled. Enable with the VM option -Darrow.memory.debug.allocator=true.
09:48:28.902 |-INFO  in c.o.a.a.m.DefaultAllocationManagerOption - allocation manager type not specified, using netty as the defaul